
# Resumen y aplicación práctica: NumPy (ndarray, broadcasting y memoria mapeada) y pandas  
**Autor:** *Romel Carlof IZAGUIRRE BARBARAN*  
**Fecha:** 2025-09-17

---

Este cuaderno recopila y comenta dos artículos fundacionales para el análisis numérico y de datos en Python:

1. **Van der Walt, Colbert & Varoquaux (2011)** — *The NumPy array: A structure for efficient numerical computation*.
2. **McKinney (2010)** — *Data Structures for Statistical Computing in Python*.

A partir de ambas referencias se muestran **resúmenes críticos** y **ejemplos reproducibles** con **NumPy** y **pandas**.



## Objetivos y metodología

**Objetivos**
- Sintetizar los aportes clave de *Van der Walt et al. (2011)* sobre `ndarray`, broadcasting y memoria mapeada.
- Sintetizar los aportes clave de *McKinney (2010)* sobre las estructuras de datos de `pandas` y su papel en la computación estadística.
- Ilustrar con **código**: (i) reglas, restricciones y casos de uso de **broadcasting** y (ii) **memoria mapeada** con `numpy.memmap`.  
- Presentar un **miniejemplo** con `pandas` que evidencie indexación, alineamiento y operaciones tipo *groupby*.

**Metodología**
- Lectura de los artículos y elaboración de un **resumen crítico**.
- Desarrollo de **experimentos breves** con `NumPy` y `pandas` para consolidar lo expuesto.
- Redacción en celdas Markdown y **código comentado** siguiendo las buenas prácticas del cuaderno.



## Artículo 1 — NumPy: ndarray, broadcasting y memoria mapeada

**Referencia resumida**  
Van der Walt, S., Colbert, S. C., & Varoquaux, G. (2011). *The NumPy array: A structure for efficient numerical computation*. **Computing in Science & Engineering, 13(2)**, 22-30.

**Ideas clave**
- `ndarray` provee **almacenamiento contiguo** (o *strided*) y un **sistema de vistas** sin copias, facilitando operaciones vectorizadas de alto rendimiento.
- El **broadcasting** permite operaciones aritméticas entre arrays de distinta forma cuando sus **dimensiones son compatibles** (regla de alinear desde la derecha y dimension 1 extensible).
- `numpy.memmap` habilita el trabajo con **datos más grandes que la memoria RAM** mediante archivos mapeados, útil para cargas que no caben en memoria a costa de latencias I/O.
- La **vectorización** y el uso de **uFuncs** minimizan bucles en Python, mejorando el rendimiento.



### Pregunta 1 — Broadcasting: explicación extendida, restricciones y ejemplos

**Reglas de compatibilidad (resumen práctico):**
1. Se comparan las formas **desde la última dimensión hacia la primera**.
2. Las dimensiones son compatibles si son **iguales** o si alguna es **1**.
3. Si alguna dimensión no cumple (ni igual ni 1), la operación **falla** con `ValueError`.

**Limitaciones y consideraciones:**
- El broadcasting **no crea copias** por sí mismo, pero la operación resultante puede materializar un array temporal grande (posible **alto consumo de memoria**).
- Operaciones que expanden implícitamente dimensiones de tamaño 1 a tamaños grandes pueden **ocupar memoria** si se materializan resultados grandes.
- No todas las funciones aceptan argumentos broadcastables en el mismo sentido; conviene **leer la docstring** y **probar dimensiones**.
- Para datos muy grandes, preferir **operaciones en bloque** (chunking) o **out=** en uFuncs cuando corresponda.

A continuación se muestran ejemplos ilustrativos.


In [ ]:

import numpy as np

# Ejemplo A: Compatibles (3, 1) con (1, 4) -> resultado (3, 4)
A = np.arange(3).reshape(3, 1)       # [[0],[1],[2]]
B = np.arange(4).reshape(1, 4)       # [[0,1,2,3]]
C = A + B
print("A.shape:", A.shape, "B.shape:", B.shape, "C.shape:", C.shape)
print(C)

# Ejemplo B: Incompatibles -> ValueError
try:
    X = np.zeros((2, 3))
    Y = np.ones((4, 1))
    Z = X + Y
except ValueError as e:
    print("\nBroadcasting incompatible:", e)

# Ejemplo C: Uso de np.newaxis / None para alinear dimensiones
v = np.array([10, 20, 30])      # shape (3,)
w = np.array([1, 2, 3, 4])      # shape (4,)

# Convertimos a (3,1) y (1,4) para obtener una tabla 3x4
tabla = v[:, None] * w[None, :]
print("\nTabla v*w shape:", tabla.shape)
print(tabla)

# Ejemplo D: ufunc con out= para evitar temporales innecesarios
out = np.empty_like(tabla)
np.multiply(v[:, None], w[None, :], out=out)
print("\nUso de out=, coincide con 'tabla':", np.allclose(out, tabla))



### Pregunta 2 — Memoria mapeada (`numpy.memmap`): eficacia y rendimiento

**Idea:** `memmap` permite tratar un archivo en disco como si fuera un array. Ventaja clave: **trabajar con datos que exceden la RAM**, cargar **trozos** sin leer todo a memoria y **persistir** parcialmente.

**Advertencias de rendimiento:**
- Para tamaños **pequeños** o acceso **aleatorio** frecuente, `memmap` puede ser **más lento** que arrays en RAM por la latencia de disco.
- Para flujos **secuenciales** grandes o procesamiento en **bloques**, `memmap` ayuda a no agotar memoria y puede ser competitivo.
- Usar `dtype` compactos (`float32`, `int32`) y **lectura por chunks** ayuda.

**Demostración breve** (no concluyente): se compara el tiempo de escritura/lectura secuencial de un array moderado con `memmap` vs. un `.npy` convencional.


In [ ]:

import numpy as np
import os, time

# Configuración de tamaño moderado para esta demo (ajustable)
shape = (5000, 200)       # ~1e6 elementos
dtype = np.float32        # ~4 MB por millón de elementos
arr = np.random.rand(*shape).astype(dtype)

# Rutas de archivos en el entorno actual
memmap_path = "demo_memmap.dat"
npy_path = "demo_array.npy"

# ---- Escritura con memmap (secuencial) ----
t0 = time.perf_counter()
mm = np.memmap(memmap_path, dtype=dtype, mode="w+", shape=shape)
mm[:] = arr[:]      # escritura secuencial
mm.flush()
t1 = time.perf_counter()

# ---- Lectura con memmap (secuencial) ----
mm_read = np.memmap(memmap_path, dtype=dtype, mode="r", shape=shape)
s1 = time.perf_counter()
_ = mm_read.sum()
s2 = time.perf_counter()

# ---- Guardar / cargar con .npy convencional ----
u0 = time.perf_counter()
np.save(npy_path, arr)
u1 = time.perf_counter()
v0 = time.perf_counter()
arr2 = np.load(npy_path, mmap_mode=None)  # carga completa en RAM
_ = arr2.sum()
v1 = time.perf_counter()

print(f"memmap write: {t1 - t0:.4f}s | memmap read&sum: {s2 - s1:.4f}s")
print(f"npy save:     {u1 - u0:.4f}s | npy load&sum:   {v1 - v0:.4f}s")

# Limpieza opcional de archivos (descomente si desea borrar al terminar)
# os.remove(memmap_path); os.remove(npy_path)



## Artículo 2 — pandas: estructuras de datos para computación estadística

**Referencia resumida**  
McKinney, W. (2010). *Data Structures for Statistical Computing in Python*. **Proceedings of the 9th Python in Science Conference**, 56-61.

**Ideas clave**
- `Series` y `DataFrame` aportan **etiquetado de ejes (índices)**, **alineamiento** automático por etiquetas y **operaciones vectorizadas** de alto nivel.
- Soporte para **lectura/escritura** de múltiples formatos, **reindexación**, **reshaping** (pivot/melt) y **groupby** (split-apply-combine).
- Integración con **NumPy** y ecosistema científico de Python.



### Pregunta 3 — Estado actual: Python vs R vs SQL vs Others (discusión breve)

- **Python (pandas, NumPy)**: gran ecosistema (scikit-learn, PyTorch, JAX), uso general, prototipado ágil. Ventajas en **integración** (APIs, web, ML). Retos: manejo de **big data** en un solo nodo; opciones modernas incluyen **Polars** y **DuckDB** para acelerar consultas tabulares locales.
- **R (tidyverse, data.table)**: ergonomía estadística, **visualización** excelente (ggplot2) y comunidad académica fuerte. *data.table* destaca por rendimiento en tablas grandes; tidyverse ofrece una **sintaxis declarativa** muy clara.
- **SQL**: estándar universal para datos tabulares y **sistemas transaccionales/analíticos**; en la práctica convive con Python/R. Herramientas como **DuckDB** hacen SQL **embebido** y eficiente en local; en la nube, motores tipo **BigQuery**/**Snowflake**.
- **Otros**: **Julia** (alto rendimiento numérico, sintaxis amigable), **Apache Arrow** (intercambio columnar de datos), **Spark** (distribuido).

**Conclusión breve:** La elección depende del **contexto**. Para análisis en un solo equipo, **pandas/Polars + DuckDB** y/o **R (data.table/tidyverse)** son opciones sólidas. Para ML, Python domina; para consultas declarativas y gobernanza de datos, **SQL** es indispensable.



### Ejemplo con `pandas`: alineamiento, groupby y pivot


In [ ]:

import pandas as pd
import numpy as np

# Dataset sintético
ventas = pd.DataFrame({
    "fecha": pd.to_datetime(["2025-01-05","2025-01-05","2025-01-06","2025-01-06","2025-01-06"]),
    "tienda": ["A","B","A","A","B"],
    "producto": ["X","X","X","Y","Y"],
    "unidades": [10, 12, 8, 5, 7],
    "precio": [2.5, 2.5, 2.5, 5.0, 5.0]
})

# Ingreso por fila
ventas["ingreso"] = ventas["unidades"] * ventas["precio"]

# GroupBy: ingreso total por tienda y producto
resumen = (
    ventas
    .groupby(["tienda", "producto"], as_index=False)["ingreso"]
    .sum()
    .sort_values(["tienda","producto"])
)

print("Ventas:
", ventas, "\n")
print("Resumen por tienda y producto:
", resumen, "\n")

# Pivot: tabla tienda x producto (ingreso)
pivot = ventas.pivot_table(index="tienda", columns="producto", values="ingreso", aggfunc="sum", fill_value=0.0)
print("Pivot tienda x producto (ingreso):\n", pivot, "\n")

# Alineamiento: suma de Series con índices distintos
s1 = pd.Series({"A": 100.0, "B": 200.0})
s2 = pd.Series({"A": 10.0, "C": 30.0})
print("Alineamiento por índice en suma s1 + s2:")
print(s1 + s2)  # 'B' y 'C' no coinciden, aparecen NaN donde falta



## Conclusiones
- `ndarray` es la piedra angular del cómputo numérico en Python; **broadcasting** potencia expresividad y velocidad, pero requiere vigilar **compatibilidad de formas** y **materialización de temporales**.
- `memmap` es útil cuando los datos **no caben en RAM** o se requiere **persistencia** incremental; su rendimiento depende de patrones de acceso y del hardware de almacenamiento.
- `pandas` aporta estructuras etiquetadas y operaciones de alto nivel que **agilizan** el análisis estadístico/tabular; conviene combinarlo con herramientas modernas (p. ej., **DuckDB/Polars**) según el tamaño/forma de los datos.



## Referencias

- Van der Walt, S., Colbert, S. C., & Varoquaux, G. (2011). The NumPy array: A structure for efficient numerical computation. *Computing in Science & Engineering, 13(2)*, 22–30. Disponible en: https://www.researchgate.net/publication/224223550_The_NumPy_Array_A_Structure_for_Efficient_Numerical_Computation

- McKinney, W. (2010). Data Structures for Statistical Computing in Python. *Proceedings of the 9th Python in Science Conference*, 56–61. Disponible en: https://www.researchgate.net/publication/265001241_Data_Structures_for_Statistical_Computing_in_Python
